# Healthcare Facility Reviews Analysis with Snowflake Cortex

This Snowflake notebook demonstrates the use of Snowflake's Cortex Generative AI functions to analyze healthcare facility reviews. The notebook showcases various AI-powered text analysis capabilities including sentiment analysis, text classification, summarization, and translation.

## Key Features

- Creates synthetic healthcare facility reviews using Cortex COMPLETE function
- Analyzes review sentiment using SENTIMENT function
- Generates summaries of reviews with SUMMARIZE function
- Classifies reviews by topic using CLASSIFY_TEXT function
- Translates reviews to other languages using TRANSLATE function
- Provides detailed recommendations based on negative reviews
- Visualizes sentiment analysis results using Streamlit

## Setup Requirements

- Snowflake database and schema for storing tables
- Streamlit for visualization components
- Python packages: pandas, plotly

## Main Components

1. **Data Generation**
   - Creates 1,000 synthetic healthcare facility reviews with `mistral-large2`
   - Uses Zipf distribution to simulate realistic data patterns

2. **Text Analysis**
   - Sentiment scoring (-1 to 1 scale)
   - Topic classification across 9 healthcare-specific categories
   - Review summarization
   - Translation capability

3. **Advanced Analysis**
   - Detailed recommendations for negative reviews
   - Analysis of sentiment patterns by topic
   - Visualization of sentiment distribution
   - Interactive dashboards using Streamlit

4. **Visualization**
   - Positive vs. negative review distribution by topic
   - Average sentiment scores
   - Review count by topic
   - Key metrics and detailed topic analysis

## Functions Used

- `snowflake.cortex.complete`
- `snowflake.cortex.sentiment`
- `snowflake.cortex.classify_text`
- `snowflake.cortex.summarize`
- `snowflake.cortex.translate`

## Tables Created

1. `FACILITY_REVIEWS`: Raw synthetic review data
2. `FACILITY_REVIEWS_ENRICHED`: Reviews enriched with AI analysis

## Usage Notes

- Initial setup may take ~5 minutes for synthetic data generation
- Uses `mistral-large2` for initial dataset setup
- Uses `mixtral-8x7b` model for detailed analysis
- Not all models are available in every Snowflake region. However you can gain access to them by enabling [Cross-Region Inference](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cross-region-inference) on your Snowflake account. (ACCOUNTADMIN role required)
- Includes custom display functions for better visualization
- Supports interactive filtering and analysis of review data



In [ ]:
# Important: Import 'plotly' from the packages menu (used at the end of the notebook)

# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

### Task-specific functions
Task-specific functions are purpose-built and managed functions that automate routine tasks, like simple summaries and quick translations, that don’t require any customization. Snowflake Cortex features are provided as **SQL functions** and are also **available in Python**. 



- `CLASSIFY_TEXT`: Given a piece of text, classifies it into one of the categories that you define.
- `EXTRACT_ANSWER`: Given a question and unstructured data, returns the answer to the question if it can be found in the data.
- `PARSE_DOCUMENT`: Given an internal or external stage with documents, returns an object that contains a JSON-formatted string with extracted text content using OCR mode, or the extracted text and layout elements using LAYOUT mode.
- `SENTIMENT`: Returns a sentiment score, from -1 to 1, representing the detected positive or negative sentiment of the given text.
- `SUMMARIZE`: Returns a summary of the given text.
- `TRANSLATE`: Translates given text from any supported language to any other.
- `EMBED_TEXT_768`: Given a piece of text, returns a vector embedding of 768 dimensions that represents that text.
- `EMBED_TEXT_1024`: Given a piece of text, returns a vector embedding of 1024 dimensions that represents that text.

In [ ]:
-- compatibility 
-- USE ROLE ACCOUNTADMIN;
-- ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

-- set your role and database
SET my_role = 'SYSADMIN';
SET my_db = 'DEMO_HC_FACILITY_REVIEWS';

-- use whatever database and schema you like, the rest of the notbook does not use fully qualified identifiers.
USE ROLE IDENTIFIER($my_role);
CREATE DATABASE IF NOT EXISTS IDENTIFIER($my_db)
    COMMENT = 'Storage and analysis of healthcare facility reviews using Snowflake Cortex AI capabilities';
USE DATABASE IDENTIFIER($my_db);
USE SCHEMA PUBLIC;

### Note 
The code in the cell below defines some custom python display functions used later in the notebook. You can choose 'collapsed' from the cell display menu, since we don't need to see this code. 

In [ ]:
# These functions are used later in the notebook but we can define them here. feel free to collapse this cell.
def display_column(df, column_name='REVIEW', title=None):
    """
    Display all entries from a specified column in a DataFrame using Streamlit components,
    including character count for each entry.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the data
    column_name (str): Name of the column to display (default: 'REVIEW')
    title (str): Title to display above the content. If None, uses column name
    """
    # Check if specified column exists
    if column_name not in df.columns:
        st.error(f"Error: DataFrame must contain a '{column_name}' column")
        return
    
    # Use column name as title if none provided
    if title is None:
        title = column_name.title()
    
    # Add a title
    st.header(title)
    
    # Display each entry in a card-like container
    for idx, entry in enumerate(df[column_name], 1):
        with st.container():
            st.markdown(f"##### {column_name} #{idx}")
            st.write(entry)
            st.metric("Characters", len(str(entry)))
            st.markdown("---")

def display_dual_columns(df, column1_name, column2_name, title=None):
    """
    Display entries from two columns side by side in a DataFrame using Streamlit components,
    including character count for each entry.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the data
    column1_name (str): Name of the first column to display
    column2_name (str): Name of the second column to display
    title (str): Title to display above the content
    """
    # Check if specified columns exist
    for col in [column1_name, column2_name]:
        if col not in df.columns:
            st.error(f"Error: DataFrame must contain a '{col}' column")
            return
    
    # Add title if provided
    if title:
        st.header(title)
    
    # Display entries side by side
    for idx in range(len(df)):
        col1, col2 = st.columns(2)
        
        # Left column
        with col1:
            st.markdown(f"##### {column1_name} #{idx + 1}")
            st.write(df[column1_name].iloc[idx])
            st.metric("Characters", len(str(df[column1_name].iloc[idx])))
            
        # Right column
        with col2:
            st.markdown(f"##### {column2_name} #{idx + 1}")
            st.write(df[column2_name].iloc[idx])
            st.metric("Characters", len(str(df[column2_name].iloc[idx])))
        
        st.markdown("---")

def display_text_and_metric(df, text_column, metric_column, title=None):
    """
    Display entries where first column is text and second column is a numeric metric,
    colored red for negative values and green for positive values.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the data
    text_column (str): Name of the column containing text
    metric_column (str): Name of the column containing numeric values
    title (str): Title to display above the content
    """
    # Check if specified columns exist
    for col in [text_column, metric_column]:
        if col not in df.columns:
            st.error(f"Error: DataFrame must contain a '{col}' column")
            return
    
    # Verify metric column contains numeric values
    if not pd.to_numeric(df[metric_column], errors='coerce').notna().all():
        st.error(f"Error: Column '{metric_column}' must contain numeric values")
        return
    
    # Add title if provided
    if title:
        st.header(title)
    
    # Display entries side by side
    for idx in range(len(df)):
        col1, col2 = st.columns(2)
        
        # Text column
        with col1:
            st.markdown(f"##### {text_column} #{idx + 1}")
            st.write(df[text_column].iloc[idx])
            st.metric("Characters", len(str(df[text_column].iloc[idx])))
            
        # Metric column with color based on value
        with col2:
            value = float(df[metric_column].iloc[idx])
            st.markdown(f"##### {metric_column} #{idx + 1}")
            if value > 0:
                st.markdown(f'<p style="color:rgb(9, 171, 59); font-size:40px; font-weight:bold">{value}</p>', unsafe_allow_html=True)
            elif value < 0:
                st.markdown(f'<p style="color:rgb(255, 43, 43); font-size:40px; font-weight:bold">{value}</p>', unsafe_allow_html=True)
            else:
                st.markdown(f'<p style="font-size:40px; font-weight:bold">{value}</p>', unsafe_allow_html=True)
        
        st.markdown("---")

# One time setup
## Let's use the Cortex COMPLETE function to generate synthetic Healthcare Facility Reviews
This will create a table with 1,000 synthetic reviews. This only needs to be run once, and may take a few minutes (~5) the first time.


### :bulb: Did you know?
Snowflake's `ZIPF` function generates random numbers that follow Zipf's law - a pattern where the frequency of any item is inversely proportional to its rank. In real life, Zipf's law appears in things like word usage (the most common word appears twice as often as the second most common, three times as often as the third, etc.). 

To generate the synthetic reviews, we leverage this to randomize parameters, but take control of the *distrubution* of them. 


In [ ]:
-- this may take a few minutes the first time you run it. ~5 min

-- choose your model, the model MUST be a constant and cannot be sourced from a column reference.
SET complete_model = 'mistral-large2';

CREATE TABLE IF NOT EXISTS FACILITY_REVIEWS 
-- CREATE OR REPLACE TABLE FACILITY_REVIEWS
AS (

-- Config 
WITH _seed as (
SELECT 
    row_number() over (order by null) as review_id,
    zipf(1,5, random()) as sentiment_seed,
    zipf(1,3, random()) as complexity_seed,
    zipf(1,4, random()) as visit_type_seed,
    zipf(1,9, random()) as core_topic_seed,
    decode(sentiment_seed,
            1, 'positive',
            2, 'neutral',
            3, 'negative',
            4, 'very negative',
            5, 'very positive') as sentiment,
    decode(complexity_seed,
            1, 'basic',
            2, 'moderate',
            3, 'detailed') as complexity,
    decode(visit_type_seed,
            1, 'routine',
            2, 'procedure',
            3, 'specialist',
            4, 'emergency') as visit_type,
    decode(core_topic_seed,
            1, 'Wait times: Time spent waiting for appointments and in-office care',
            2, 'Staff interactions: How personnel treat and engage with patients',
            3, 'Medical care quality: Effectiveness and competence of treatment',
            4, 'Facility conditions: Building cleanliness and maintenance',
            5, 'Administrative processes: Efficiency of paperwork and procedures',
            6, 'Communication: Clarity and completeness of information sharing',
            7, 'Billing/insurance: Handling of payments and claims',
            8, 'Follow-up care: Post-treatment support and monitoring',
            9, 'Food quality: Quality and variety of cafeteria or vending options') as core_topic,            
FROM table(generator(rowcount => 1000))
)
-- Output
SELECT
    review_id,
    sentiment,
    complexity,
    visit_type,
    core_topic,
$$
# SYSTEM CONTEXT
You are an advanced testing tool designed for generating diverse, coherent patient reviews.

# INPUT PARAMETERS
review_config: {
    sentiment: $$ || sentiment || $$,
    visit_type: $$ || visit_type || $$,
    complexity: $$ || complexity || $$,
    core_topic: $$ || core_topic || $$,
}

# OUTPUT SPECIFICATION
- Generate ONLY the text string of the patient review based on the review_config INPUT PARAMETERS
- Review should be between 5 and 30 setences

# CONSTRAINTS
- Keep medical scenarios realistic but generic
- Include natural language patterns
- Avoid specific names, locations, or identifiable details
- Generate clinically plausible scenarios
- Do not use the word "overall"
- Do not start or end the response with '"'

Return only the review, no additional text or explanations.
$$ as prompt_string,
    object_construct('role', 'user',
                     'content', prompt_string) as prompt_object,
    array_construct(prompt_object) as complete_prompt,
    snowflake.cortex.complete($complete_model, complete_prompt, {'temperature': 1.0}) as cortex_reponse,
    trim(cortex_reponse:choices[0]:messages::string) as review,
    cortex_reponse:usage:model::varchar(100) as model,
    cortex_reponse:usage:completion_tokens::int as completion_tokens,
    cortex_reponse:usage:prompt_tokens::int as prompt_tokens,
    cortex_reponse:usage:total_tokens::int as total_tokens,
    cortex_reponse:created::timestamp_ntz as created_utc_ts,
FROM _seed);

In [ ]:
SELECT 
    REVIEW_ID,
    REVIEW,
FROM FACILITY_REVIEWS
SAMPLE (10 ROWS);

## SENTIMENT
The SENTIMENT function returns sentiment as a score between -1 to 1 

With -1 being the most negative and 1 the most positive, with values around 0 neutral) for the given English-language input text.

In [ ]:
-- random sample of positive and negative reviews
SELECT 
    REVIEW_ID,
    REVIEW,
    SNOWFLAKE.CORTEX.SENTIMENT(REVIEW) as SENTIMENT_SCORE,
FROM FACILITY_REVIEWS
SAMPLE (2 ROWS)
WHERE SENTIMENT_SCORE > 0
UNION ALL
SELECT 
    REVIEW_ID,
    REVIEW,
    SNOWFLAKE.CORTEX.SENTIMENT(REVIEW) as SENTIMENT_SCORE,
FROM FACILITY_REVIEWS
SAMPLE (2 ROWS)
WHERE SENTIMENT_SCORE < 0;

In [ ]:
setntiment_df = cells.sample_sentiment_sql.to_pandas()

display_text_and_metric(setntiment_df, 'REVIEW', 'SENTIMENT_SCORE', title="Reviews")

## SUMMARIZE
The `SUMMARIZE` function returns a summary of the given English text.

In [ ]:
SELECT 
    REVIEW_ID,
    REVIEW,
    SNOWFLAKE.CORTEX.SUMMARIZE(REVIEW) as SUMMARY,
FROM FACILITY_REVIEWS
SAMPLE (3 ROWS);

In [ ]:
summary_df = cells.sample_summary_sql.to_pandas()

display_dual_columns(summary_df, 'REVIEW', 'SUMMARY', 'Review and Summary')

## CLASSIFY_TEXT
The `CLASSIFY_TEXT` function classifies free-form text into categories that you provide. The text may be a plain English string.

In [ ]:
SELECT 
    REVIEW_ID,
    REVIEW,
    SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
        REVIEW,
        [
            {'label': 'Wait times',
             'description': 'Time spent waiting for appointments and in-office care'
            },
            {'label': 'Staff interactions',
             'description': 'How personnel treat and engage with patients'
            },
            {'label': 'Medical care quality',
             'description': 'Effectiveness and competence of treatment'
            },
            {'label': 'Facility conditions',
             'description': 'Building cleanliness and maintenance'
            },
            {'label': 'Administrative processes',
             'description': 'Efficiency of paperwork and procedures'
            },
            {'label': 'Communication',
             'description': 'Clarity and completeness of information sharing'
            },
            {'label': 'Billing/insurance',
             'description': 'Handling of payments and claims'
            },
            {'label': 'Follow-up care',
             'description': 'Post-treatment support and monitoring'
            },
            {'label': 'Food quality',
             'description': 'Quality and variety of cafeteria or vending options'
            }
        ],
            {'task_description': 'Return a classification of the primary topic of the review'}
        ):label::string as topic_classification
FROM FACILITY_REVIEWS
SAMPLE (10 ROWS);

## TRANSLATE
The `TRANSLATE` function translates text from the indicated or detected source language to a target language.

In [ ]:
SELECT 
    REVIEW_ID,
    REVIEW,
    SNOWFLAKE.CORTEX.TRANSLATE(REVIEW,'en','fr') as REVIEW_TRANSLATED,
FROM FACILITY_REVIEWS
SAMPLE (2 ROWS);

In [ ]:
translate_df = cells.sample_translate_sql.to_pandas()

display_dual_columns(translate_df, 'REVIEW', 'REVIEW_TRANSLATED', 'Review and Spanish Translation')

## Let's tie together by creating a new table of our reviews, enriched with with data from Cortex LLM functions!

In [ ]:
CREATE TABLE IF NOT EXISTS FACILITY_REVIEWS_ENRICHED 
-- CREATE OR REPLACE TABLE FACILITY_REVIEWS_ENRICHED 
AS (
SELECT 
    REVIEW_ID,
    REVIEW,
    SNOWFLAKE.CORTEX.SUMMARIZE(REVIEW) as SUMMARY,
    SNOWFLAKE.CORTEX.SENTIMENT(REVIEW) as SENTIMENT_SCORE,
    SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
        REVIEW,
        [
            {'label': 'Wait times',
             'description': 'Time spent waiting for appointments and in-office care'
            },
            {'label': 'Staff interactions',
             'description': 'How personnel treat and engage with patients'
            },
            {'label': 'Medical care quality',
             'description': 'Effectiveness and competence of treatment'
            },
            {'label': 'Facility conditions',
             'description': 'Building cleanliness and maintenance'
            },
            {'label': 'Administrative processes',
             'description': 'Efficiency of paperwork and procedures'
            },
            {'label': 'Communication',
             'description': 'Clarity and completeness of information sharing'
            },
            {'label': 'Billing/insurance',
             'description': 'Handling of payments and claims'
            },
            {'label': 'Follow-up care',
             'description': 'Post-treatment support and monitoring'
            },
            {'label': 'Food quality',
             'description': 'Quality and variety of cafeteria or vending options'
            }
        ],
            {'task_description': 'Return a classification of the primary topic of the review'}
        ):label::string as topic_classification
FROM FACILITY_REVIEWS
);

## Define Prompt

Let's create a prompt to analyse a review, and identify key issues and provide recomendations for remediation.

*Note: In a real world use case, you would not likely do this for each individual reivew, but rather over the all reviews in a time frame.*

For an example see this video:
- [How To Analyze Support Tickets With Snowflake Cortex LLMs, Notebooks, And Streamlit](https://www.youtube.com/watch?v=tpKqZO7kTf8)

In [ ]:
recommendation_prompt = """$$
You are a healthcare facility operations consultant specializing in patient experience optimization. Your task is to analyze patient reviews and provide actionable recommendations for facility owners.

When analyzing a review, follow these steps:

1. First, carefully identify explicit and implicit pain points in the patient's review. Consider:
   - Wait times
   - Staff interactions
   - Facility conditions
   - Communication issues
   - Administrative processes
   - Medical care quality
   - Follow-up care
   - Billing and insurance handling

2. For each identified issue:
   - Assess its severity
   - Consider its potential impact on other patients
   - Evaluate how it affects the facility's reputation
   - Determine if it presents any regulatory or compliance risks

3. Generate specific, actionable recommendations that:
   - Address the root cause of each issue
   - Can be implemented within 3-6 months
   - Consider resource constraints
   - Align with healthcare regulations
   - Prioritize patient safety and satisfaction

Format your response as follows:

# Key Issues Identified
- [List the main problems identified in the review]

# Priority Recommendations
1. [First recommendation]
   - Expected impact
   - Required resources

2. [Second recommendation]
   - Expected impact
   - Required resources

[Continue with additional recommendations]

# Quick Wins
- [List 2-3 immediate actions that can be taken within 1 week]

# Long-term Considerations
- [List any systemic issues that need strategic planning]

Guidelines for recommendations:
- Be specific and actionable
- Include estimated implementation costs when relevant
- Consider staff training needs
- Factor in regulatory compliance
- Focus on sustainable solutions
- Prioritize based on impact vs. effort

Remember to:
- Maintain a professional, solution-focused tone
- Consider both patient and staff perspectives
- Include measurable outcomes
- Suggest monitoring mechanisms for implemented changes

Example Review:
"Had to wait 2 hours to see the doctor. The reception staff were rude and didn't acknowledge me when I checked in. The facility was clean but outdated. The doctor was great once I saw them, but getting any follow-up information has been impossible. Leaving messages but no one calls back."

Example Response:

# Key Issues Identified
- Excessive wait times
- Poor front desk customer service
- Communication breakdown in follow-up care
- Outdated facility appearance

# Priority Recommendations
1. Implement digital queue management system
   - Impact: 40% reduction in wait times
   - Resources: Software system, staff training

2. Customer service training program for front desk staff
   - Impact: Improved patient satisfaction scores
   - Resources: External trainer, weekly sessions

3. Establish dedicated follow-up care coordinator
   - Impact: 24-hour response guarantee
   - Resources: New hire, communication protocols

# Quick Wins
- Install self-check-in kiosk
- Create standardized follow-up call schedule
- Post visible wait time estimates

# Long-term Considerations
- Facility modernization plan
- Staff retention strategy
- Patient communication system upgrade

Please analyze the provided review and generate recommendations following this format.

## REVIEW

$$
"""

### With our prompt crafted, let's have Cortex analyse our some of reviews with the most negative sentiment.

In [ ]:
SELECT 
    REVIEW_ID,
    REVIEW,
    SENTIMENT_SCORE,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-3-5-sonnet',
        CONCAT(
            {{recommendation_prompt}}, 
            REVIEW
            )
        ) as recommendations
FROM FACILITY_REVIEWS_ENRICHED
SAMPLE (2 rows)
WHERE SENTIMENT_SCORE < -0.5;

In [ ]:
recommendation_df = cells.recommendation_sql.to_pandas()

display_dual_columns(recommendation_df, 'REVIEW', 'RECOMMENDATIONS', 'Negative Reviews and Recommendations')

## Let's see the results of Cortex in action for a sentiment analysis use case

* First will will see how an analyst can use the new extracted attributes to filter the review data set
* Then we will see how we can use these attrubutes to perform analysis over the entire data set.



In [ ]:
-- filter by Communication Issues with a Sentiment Score of less than -.05
SELECT
    *
FROM FACILITY_REVIEWS_ENRICHED
WHERE TRUE
    AND topic_classification = 'Communication'
    AND sentiment_score < -0.5;

In [ ]:
SELECT 
    topic_classification,
    count(topic_classification) as topic_count,
    (count_if(sentiment_score > 0)/topic_count * 100)::decimal(8,2) as prct_positive,
    (count_if(sentiment_score < 0)/topic_count * 100)::decimal(8,4) as prct_negative,
    min(sentiment_score)::decimal(8,4)*100 as min_sentiment_score,
    max(sentiment_score)::decimal(8,4)*100 as max_sentiment_score,
    avg(sentiment_score)::decimal(8,4)*100 as avg_sentiment_score,
    array_agg(sentiment_score*100) as sentiment_scores
FROM FACILITY_REVIEWS_ENRICHED
group by all;

## Since Snowflake Notebooks have Streamlit built-in, we can visulize the results! :sparkles:
For the cell below, choose 'results only' for the cell display menu so the python code to make the chart is distracting.

In [ ]:
import streamlit as st
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_sentiment_visualization(df):
    """
    Create a sentiment analysis visualization using Streamlit and Plotly.
    
    Parameters:
    df: pandas DataFrame with columns:
        - TOPIC_CLASSIFICATION
        - TOPIC_COUNT
        - PRCT_POSITIVE
        - PRCT_NEGATIVE
        - AVG_SENTIMENT_SCORE
    """
    # Sort DataFrame by negative percentage for first chart
    df_neg_sorted = df.sort_values('PRCT_NEGATIVE', ascending=False)
    
    # Sort DataFrame by count for the count chart
    df_count_sorted = df.sort_values('TOPIC_COUNT', ascending=False)
    
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=("Positive vs Negative Reviews by Topic", 
                       "Average Sentiment Score by Topic",
                       "Number of Reviews by Topic"),
        row_heights=[0.4, 0.3, 0.3],
        vertical_spacing=0.1
    )
    
    # Add bars for positive percentages
    fig.add_trace(
        go.Bar(
            name="Positive",
            x=df_neg_sorted['TOPIC_CLASSIFICATION'],
            y=df_neg_sorted['PRCT_POSITIVE'],
            marker_color='#2ecc71',
            text=df_neg_sorted['PRCT_POSITIVE'].round(1).astype(str) + '%',
            textposition='auto',
        ),
        row=1, col=1
    )
    
    # Add bars for negative percentages
    fig.add_trace(
        go.Bar(
            name="Negative",
            x=df_neg_sorted['TOPIC_CLASSIFICATION'],
            y=df_neg_sorted['PRCT_NEGATIVE'],
            marker_color='#e74c3c',
            text=df_neg_sorted['PRCT_NEGATIVE'].round(1).astype(str) + '%',
            textposition='auto',
        ),
        row=1, col=1
    )
    
    # Add average sentiment score bar chart
    fig.add_trace(
        go.Bar(
            name="Average Sentiment",
            x=df_neg_sorted['TOPIC_CLASSIFICATION'],
            y=df_neg_sorted['AVG_SENTIMENT_SCORE'],
            marker_color='#3498db',
            text=df_neg_sorted['AVG_SENTIMENT_SCORE'].round(2).astype(str),
            textposition='auto',
        ),
        row=2, col=1
    )
    
    # Add review count bar chart
    fig.add_trace(
        go.Bar(
            name="Review Count",
            x=df_count_sorted['TOPIC_CLASSIFICATION'],
            y=df_count_sorted['TOPIC_COUNT'],
            marker_color='#9b59b6',
            text=df_count_sorted['TOPIC_COUNT'].astype(str),
            textposition='auto',
        ),
        row=3, col=1
    )
    
    # Update layout
    fig.update_layout(
        barmode='group',
        height=1000,
        showlegend=True,
        yaxis_title="Percentage of Reviews",
        yaxis2_title="Average Sentiment Score",
        yaxis3_title="Number of Reviews"
    )
    
    # Display the plot
    st.plotly_chart(fig, use_container_width=True)
    
    # Display additional metrics
    st.subheader("Detailed Topic Analysis")
    
    # Create four columns for metrics
    cols = st.columns(4)
    
    # Calculate metrics
    best_topic = df.loc[df['PRCT_POSITIVE'].idxmax(), 'TOPIC_CLASSIFICATION']
    highest_positive = df['PRCT_POSITIVE'].max()
    worst_topic = df.loc[df['PRCT_NEGATIVE'].idxmax(), 'TOPIC_CLASSIFICATION']
    highest_negative = df['PRCT_NEGATIVE'].max()
    best_avg = df.loc[df['AVG_SENTIMENT_SCORE'].idxmax(), 'TOPIC_CLASSIFICATION']
    highest_avg = df['AVG_SENTIMENT_SCORE'].max()
    most_reviewed = df.loc[df['TOPIC_COUNT'].idxmax(), 'TOPIC_CLASSIFICATION']
    highest_count = df['TOPIC_COUNT'].max()
    
    # Display metrics
    with cols[0]:
        st.metric(
            "Most Positive Topic",
            best_topic,
            f"{highest_positive:.1f}% Positive"
        )
    
    with cols[1]:
        st.metric(
            "Most Negative Topic",
            worst_topic,
            f"{highest_negative:.1f}% Negative"
        )
    
    with cols[2]:
        st.metric(
            "Highest Average Sentiment",
            best_avg,
            f"{highest_avg:.2f}"
        )
    
    with cols[3]:
        st.metric(
            "Most Reviewed Topic",
            most_reviewed,
            f"{highest_count} reviews"
        )
    
    # Display raw data in an expandable section
    with st.expander("View Raw Data"):
        st.dataframe(df)

# Display Results
analysis_df = cells.example_analysis_sql.to_pandas()
st.title("Facility Reviews Sentiment Analysis")
create_sentiment_visualization(analysis_df)

# COMPLETE

Given a prompt, the instruction-following `COMPLETE` function generates a response using your choice of language model. In the simplest use case, the prompt is a single string. 

You may also provide a conversation including multiple prompts and responses for interactive chat-style usage, and in this form of the function you can also specify hyperparameter options to customize the style and size of the output. 

In order to implement safeguards, you can also enable the Cortex Guard parameter that filters potentially unsafe and harmful responses from a LLM.

At the time of this writing, The `COMPLETE` function supports the following models. More models are added added over time. Different models can have different costs:

#### [Currently Available Models](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex#arguments)


## We can also use Cortex to create a report based on the above data. 
For this use case, we will leverage the Python API for Cortex as opposed to SQL

In [ ]:
# import complete from the snowflake-python-ml package
from snowflake.cortex import complete

# reference the previous SQL result
df = cells.example_analysis_sql.to_pandas()
# remove array of scores
df = df.drop('SENTIMENT_SCORES', axis=1)

# craft prompt for the llm
prompt = '''
Analyze this healthcare facility sentiment dataset containing pre-calculated average scores per topic (scale: [-100-100]) and:

1. **Score Benchmarking**
   - Compare each topic's score to:
     * Industry benchmarks ([specify if available])
     * Ideal performance thresholds ([Y])
   - Flag topics where:
     * Score < [threshold1] (critical)
     * Score between [threshold1] and [threshold2] (needs improvement)
     * Score > [threshold2] (success)

3. **Topic Interdependencies**
   - Perform quadrant analysis:
     Y-axis: Topic importance (predefined or frequency)
     X-axis: Current performance score
   - Identify:
     * High importance/Low performance (Urgent focus)
     * High importance/High performance (Maintain)
     * Low importance/High performance (Optimize resources)
     * Low importance/Low performance (Deprioritize)

4. Output should be plain language with recomendations

5. Do not provide followup questions to the user. 

6. Output should be in markdown format with headers, subheaders, and bulletpoints.

'''

# We will provide a prompt and the aggregated dataset from above
analysis_explain = complete('claude-3-5-sonnet', f'{prompt}: {df}')

#display report
st.markdown(analysis_explain)